In [2]:
!pip install prophet

  Obtaining dependency information for prophet from https://files.pythonhosted.org/packages/41/46/75309abde08c10f9be78bcfca581be430b5d8303d847de8d88190f4d5c21/prophet-1.1.6-py3-none-macosx_10_11_x86_64.whl.metadata
  Obtaining dependency information for cmdstanpy>=1.0.4 from https://files.pythonhosted.org/packages/2a/80/04b3989f055e555737b3b6944f02112e90cfc4f2e53c763771dded22e684/cmdstanpy-1.2.5-py3-none-any.whl.metadata
  Obtaining dependency information for holidays<1,>=0.25 from https://files.pythonhosted.org/packages/65/9c/01da17421bd6a0aa230839b8313fa1615c81555f94954350fca4f4b7f129/holidays-0.73-py3-none-any.whl.metadata
  Obtaining dependency information for importlib-resources from https://files.pythonhosted.org/packages/a4/ed/1f1afb2e9e7f38a545d628f864d562a5ae64fe6f7a10e28ffb9b185b4e89/importlib_resources-6.5.2-py3-none-any.whl.metadata
  Obtaining dependency information for stanio<2.0.0,>=0.4.0 from https://files.pythonhosted.org/packages/56/87/37a80e4d5bd453c33262d8fb618b6840

In [31]:
import pandas as pd
from prophet import Prophet
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import plotly.graph_objects as go

# Load dataset
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv')

df.columns = ['ds', 'y']
df['ds'] = pd.to_datetime(df['ds'])

# Split data into train and test
train_df = df[:-12]
test_df = df[-12:]

# Prophet Forecast
prophet_model = Prophet()
prophet_model.fit(train_df)
future = prophet_model.make_future_dataframe(periods=12, freq='M')
prophet_forecast = prophet_model.predict(future)[['ds', 'yhat']]
prophet_forecast.set_index('ds', inplace=True)

# ETS Forecast
ets_model = ExponentialSmoothing(
    train_df['y'],
    trend='add',
    seasonal='add',
    seasonal_periods=12
)
ets_fit = ets_model.fit()
ets_forecast = pd.DataFrame({
    'ds': test_df['ds'].values,
    'yhat': ets_fit.forecast(12).values
}).set_index('ds')

# Create Plotly Figure
fig = go.Figure()

# Actuals
fig.add_trace(go.Scatter(
    x=df['ds'], y=df['y'],
    mode='lines+markers',
    name='Actuals',
    line=dict(color='black', width=2),
    marker=dict(size=4)
))

# Prophet Forecast
fig.add_trace(go.Scatter(
    x=prophet_forecast.index, y=prophet_forecast['yhat'],
    mode='lines',
    name='Prophet Forecast',
    line=dict(color='blue', width=2)
))

# ETS Forecast
fig.add_trace(go.Scatter(
    x=ets_forecast.index, y=ets_forecast['yhat'],
    mode='lines',
    name='Baseline ETS Forecast',
    line=dict(color='orange', width=2, dash='dash')
))

# Vertical line for forecast start
forecast_start = test_df['ds'].iloc[0].timestamp() * 1000  # milliseconds since epoch for Plotly

fig.add_vline(
    x=forecast_start,
    line_width=2, line_dash="dot", line_color="gray",
    annotation_text="Forecast Start", annotation_position="top right"
)

# Layout settings
fig.update_layout(
    title='📈 Forecast Comparison: Prophet vs ETS',
    xaxis_title='Date',
    yaxis_title='Number of Passengers',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.5)', bordercolor='black'),
    template='plotly_white',
    height=500
)

fig.show()


22:38:30 - cmdstanpy - INFO - Chain [1] start processing
22:38:30 - cmdstanpy - INFO - Chain [1] done processing


In [41]:
import pandas as pd
from prophet import Prophet
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import plotly.graph_objects as go

# Load dataset
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv')

df.columns = ['ds', 'y']
df['ds'] = pd.to_datetime(df['ds'])

# Split data into train and test
train_df = df[:-12]
test_df = df[-12:]

# Prophet Forecast
prophet_model = Prophet()
prophet_model.fit(train_df)
future = prophet_model.make_future_dataframe(periods=12, freq='M')
prophet_forecast = prophet_model.predict(future)

# Set index for easier slicing
prophet_forecast.set_index('ds', inplace=True)

# ETS Forecast
ets_model = ExponentialSmoothing(
    train_df['y'],
    trend='add',
    seasonal='add',
    seasonal_periods=12
)
ets_fit = ets_model.fit()
ets_forecast = pd.DataFrame({
    'ds': test_df['ds'].values,
    'yhat': ets_fit.forecast(12).values
}).set_index('ds')

# Create Plotly Figure
fig = go.Figure()


# Only show actuals up to the last date in test set (i.e., no future leakage)
actuals_end = test_df['ds'].max()
actuals_df = df[df['ds'] <= actuals_end]
actuals_df=actuals_df[actuals_df['ds']<'1960-01-01']

fig.add_trace(go.Scatter(
    x=actuals_df['ds'], y=actuals_df['y'],
    mode='lines+markers',
    name='Actuals',
    line=dict(color='black', width=2),
    marker=dict(size=4)
))

# Prophet Forecast line
fig.add_trace(go.Scatter(
    x=prophet_forecast.index, y=prophet_forecast['yhat'],
    mode='lines',
    name='Prophet Forecast',
    line=dict(color='blue', width=2)
))

# Prophet Confidence Interval (shaded area)

# Prophet Confidence Interval (shaded area)
fig.add_trace(go.Scatter(
    x=pd.concat([
        pd.Series(prophet_forecast.index),                # Forward dates
        pd.Series(prophet_forecast.index[::-1])           # Reverse for fill
    ]),
    y=pd.concat([
        prophet_forecast['yhat_upper'],                   # Upper bound
        prophet_forecast['yhat_lower'][::-1]              # Lower bound reversed
    ]),
    fill='toself',
    fillcolor='rgba(0, 0, 255, 0.2)',  # translucent blue
    line=dict(color='rgba(255,255,255,0)'),  # invisible line
    hoverinfo="skip",
    showlegend=True,
    name='Prophet Confidence Interval'
))



# ETS Forecast
fig.add_trace(go.Scatter(
    x=ets_forecast.index, y=ets_forecast['yhat'],
    mode='lines',
    name='Baseline ETS Forecast',
    line=dict(color='orange', width=2, dash='dash')
))

# Vertical line for forecast start
forecast_start = test_df['ds'].iloc[0].timestamp() * 1000  # milliseconds since epoch for Plotly

fig.add_vline(
    x=forecast_start,
    line_width=2, line_dash="dot", line_color="gray",
    annotation_text="Forecast Start", annotation_position="top right"
)

# Layout settings
fig.update_layout(
    title='📈 Forecast Comparison: Prophet vs ETS with Confidence Interval',
    xaxis_title='Date',
    yaxis_title='Number of Passengers',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.5)', bordercolor='black'),
    template='plotly_white',
    height=500
)

fig.show()


22:46:16 - cmdstanpy - INFO - Chain [1] start processing
22:46:16 - cmdstanpy - INFO - Chain [1] done processing


In [48]:
import pandas as pd
from prophet import Prophet
import plotly.graph_objects as go

# Load dataset
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv')
df.columns = ['ds', 'y']
df['ds'] = pd.to_datetime(df['ds'])

# Split data into train and test
train_df = df[:-12]
test_df = df[-12:]

# Prophet Forecast
prophet_model = Prophet()
prophet_model.fit(train_df)
future = prophet_model.make_future_dataframe(periods=12, freq='M')
prophet_forecast = prophet_model.predict(future)
prophet_forecast.set_index('ds', inplace=True)

# Rolling Moving Average Forecast
window = 12
rolling_forecast_values = list(train_df['y'].iloc[-window:].values)  # start with last 5 real values
forecast_dates = test_df['ds'].values
rolling_preds = []

for i in range(len(forecast_dates)):
    avg = sum(rolling_forecast_values[-window:]) / window
    rolling_preds.append(avg)
    rolling_forecast_values.append(avg)  # append prediction for use in next rolling window

# Create DataFrame
rolling_forecast = pd.DataFrame({
    'ds': forecast_dates,
    'yhat': rolling_preds
}).set_index('ds')

# ------------------ Plotting ------------------ #
fig = go.Figure()

# Plot Actuals
actuals_end = test_df['ds'].max()
actuals_df = df[df['ds'] <= actuals_end]
actuals_df = actuals_df[actuals_df['ds'] < '1960-01-01']

fig.add_trace(go.Scatter(
    x=actuals_df['ds'], y=actuals_df['y'],
    mode='lines+markers',
    name='Actuals',
    line=dict(color='black', width=2),
    marker=dict(size=4)
))

# Prophet Forecast
fig.add_trace(go.Scatter(
    x=prophet_forecast.index, y=prophet_forecast['yhat'],
    mode='lines',
    name='Prophet Forecast',
    line=dict(color='blue', width=2)
))

# Prophet Confidence Interval
fig.add_trace(go.Scatter(
    x=pd.concat([
        pd.Series(prophet_forecast.index),
        pd.Series(prophet_forecast.index[::-1])
    ]),
    y=pd.concat([
        prophet_forecast['yhat_upper'],
        prophet_forecast['yhat_lower'][::-1]
    ]),
    fill='toself',
    fillcolor='rgba(0, 0, 255, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='Prophet Confidence Interval'
))

# Rolling Moving Average Forecast
fig.add_trace(go.Scatter(
    x=rolling_forecast.index, y=rolling_forecast['yhat'],
    mode='lines',
    name='Rolling Moving Average Forecast',
    line=dict(color='green', width=2, dash='dot')
))

# Forecast start vertical line
forecast_start = test_df['ds'].iloc[0].timestamp() * 1000
fig.add_vline(
    x=forecast_start,
    line_width=2, line_dash="dot", line_color="gray",
    annotation_text="Forecast Start", annotation_position="top right"
)

# Layout settings
fig.update_layout(
    title='📈 Forecast Comparison: Prophet vs Rolling Moving Average',
    xaxis_title='Date',
    yaxis_title='Number of Passengers',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.5)', bordercolor='black'),
    template='plotly_white',
    height=500
)

fig.show()


22:52:45 - cmdstanpy - INFO - Chain [1] start processing
22:52:45 - cmdstanpy - INFO - Chain [1] done processing


In [46]:
rolling_forecast

,yhat
ds,
1960-01-01,428.333333
1960-02-01,428.333333
1960-03-01,428.333333
1960-04-01,428.333333
1960-05-01,428.333333
1960-06-01,428.333333
1960-07-01,428.333333
1960-08-01,428.333333
1960-09-01,428.333333


In [44]:
sma_forecast

,yhat
ds,
1960-01-01,428.333333
1960-02-01,428.333333
1960-03-01,428.333333
1960-04-01,428.333333
1960-05-01,428.333333
1960-06-01,428.333333
1960-07-01,428.333333
1960-08-01,428.333333
1960-09-01,428.333333
